In [ ]:
!pip install rdkit
!pip install tqdm
!pip install e3nn
!pip install smarts
!pip install torch_geometric
!pip install torch-cluster==1.6.3 -f https://data.pyg.org/whl/torch-2.2.1+cu121.html
!pip install torch-scatter==2.1.2 -f https://data.pyg.org/whl/torch-2.2.1+cu121.html

In [ ]:
import warnings

# Disable all UserWarning warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Only Run the Following on Colab

In [ ]:
from IPython import get_ipython
import os
import sys

if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive')

  base_dir = "/content/drive/MyDrive/Colab Notebooks/DetaNet/code"
  src_dir = "/content/drive/MyDrive/Colab Notebooks/AISpec/src"
  sys.path.append(base_dir)
  sys.path.append(src_dir)
  os.chdir(base_dir)
else:
  pass

In [ ]:
import logging
import sys
import warnings
warnings.filterwarnings('ignore')

# Create logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Create STDERR handler
handler = logging.StreamHandler(sys.stdout)

# Create formatter and add it to the handler
formatter = logging.Formatter('%(asctime)-15s %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

# Set STDERR handler as the only handler
logger.handlers = [handler]

In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from detanet_model import *
from torch_geometric.loader import DataLoader

'''First, the pytorch library can be used to load the dataset, which consists of 130K molecules, after importing the pytorch library.'''
dataset=torch.load('../data/qm9s.pt')

In [ ]:
len(dataset)

129817

In [ ]:
'''where each molecule is a gemetric.data.Data. containing coordinates, atomic numbers, edge indices, and various properties.'''
dataset[0]

Data(edge_index=[2, 20], pos=[5, 3], number=1, smile='C', z=[5], quadrupole=[1, 3, 3], octapole=[1, 3, 3, 3], npacharge=[5], dipole=[1, 3], polar=[1, 3, 3], hyperpolar=[1, 3, 3, 3], energy=[1, 1], Hij=[20, 3, 3], Hi=[5, 3, 3], dedipole=[5, 3, 3], depolar=[5, 3, 6], tran_dipole=[1, 10, 3], tran_energy=[1, 10])

# Build Databaaes

In [ ]:
from ir_dataset import IrDataset, ToTransmittance

ir_ds = IrDataset(dataset, ir_sigma=5,
                xir=torch.linspace(50, 4000, 3600),
                xuv=torch.linspace(100, 700, 240),
                xnmr=torch.linspace(0, 12, 516),
                data_path="../data")
ir_ds.cal_ir()
ir_ds.cal_uv()
ir_ds.cal_nmr()
ir_ds.save()

 86%|████████▌ | 111686/129817 [100:43:19<34:23:51,  6.83s/it]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

100%|██████████| 129817/129817 [129:05:09<00:00,  3.58s/it]   


In [ ]:
!ls -l ../data

total 5273716
-rw-r--r-- 1 jupyter jupyter 1850204752 Apr  6 05:27 ir_features.pth
-rw-r--r-- 1 jupyter jupyter   67672534 Apr  6 05:27 ir_labels.pth
-rw-r--r-- 1 jupyter jupyter  305382468 Apr 30 03:17 nmr_features.pth
-rw-r--r-- 1 jupyter jupyter   68191818 Apr 30 03:17 nmr_labels.pth
-rw-r--r-- 1 jupyter jupyter 2876992905 Mar 31 18:38 qm9s.pt
-rw-r--r-- 1 jupyter jupyter       3272 Apr 28 19:26 readme.txt
-rw-r--r-- 1 jupyter jupyter  163752103 Apr 30 03:16 uv_features.pth
-rw-r--r-- 1 jupyter jupyter   68061997 Apr 30 03:16 uv_labels.pth


# Read Data Back

In [ ]:
from ir_dataset import IrDataset

ds = IrDataset(data_list=None, data_path="../data")
ds.load()

2024-05-03 15:51:02,618 root - INFO - Loading ir feature data from ../data/ir_features.pth...
2024-05-03 15:51:08,347 root - INFO - Loading ir label data from ../data/ir_labels.pth...
2024-05-03 15:51:14,408 root - INFO - Loading uv feature data from ../data/uv_features.pth...
2024-05-03 15:51:21,024 root - INFO - Loading uv label data from ../data/uv_labels.pth...
2024-05-03 15:51:25,075 root - INFO - Loading nmr feature data from ../data/nmr_features.pth...
2024-05-03 15:51:32,131 root - INFO - Loading nmr label data from ../data/nmr_labels.pth...
2024-05-03 15:51:36,237 root - INFO - Consolidating data...
2024-05-03 15:51:40,710 root - INFO - Removing 2307 bad records from data!!!


In [ ]:
len(ds)

127510